In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [5]:
import requests

def download_excel_file(url, save_path):
    """
    Downloads an Excel file from the given URL and saves it locally.
    
    :param url: The URL of the Excel file to download.
    :param save_path: The local path where the file will be saved.
    """
    print(f"Downloading file from: {url}")
    
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Write the file content to the specified save path
            with open(save_path, "wb") as file:
                file.write(response.content)
            print(f"File successfully saved as: {save_path}")
        else:
            print(f"Failed to download the file. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while downloading the file: {e}")

# Main program
if __name__ == "__main__":
    # Specify the URL of the Excel file (replace this with the actual URL)
    excel_url = "https://docs.google.com/spreadsheets/d/<GoogleSheetID>/export?format=xlsx"
    
    # Specify the local path where the file will be saved
    save_path = "Survivor_S32_Kaoh_Rong.xlsx"
    
    # Call the function to download the Excel file
    download_excel_file(excel_url, save_path)

Failed to download the file. Status code: 404
